# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import logging 
import os
import csv

from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd

import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

print("SDK Version:",azureml.core.VERSION)

SDK Version: 1.18.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
found = False

key = "house_sale"
description_text = "house sales prediction dataset"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:

    data = 'https://www.kaggle.com/harlfoxem/housesalesprediction/download'
    dataset = Dataset.Tabular.from_delimited_files(data)
    dataset = dataset.register(workspace=ws,name = key,description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Avinash-AutoML'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FP5NT6YGU to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [10]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aml_compute_cluster = "Avinash-Test"

try:
    compute_target = ComputeTarget(workspace=ws,name = aml_compute_cluster)
    print('Found Exisitng cluster, use it ')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes = 4,)
    compute_target = ComputeTarget.create(ws,aml_compute_cluster)


compute_target.wait_for_completion(show_output=True)

Found Exisitng cluster, use it 
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
found = False

key = "house_sales"
description_text = "house sales prediction dataset"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(df,test_size = 0.2,random_state = 42)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping": True,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = 'regression',
    debug_log = 'automl_reg_errors.log',
    training_data = x_train,
    label_column_name = "price",
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run,fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#TODO: Save the best model

best_run.register_model(model_path = './outputs',model_name = 'BestAutoML_model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service